TARGET LABEL: Helpful_Votes
SCALER: STANDARD SCALER
ANN'S : 
    Combination 1: 7-100-100-1
    
    Combination 2:  7-4-4-1

    Combination 3:  7-50-100-200-100-50-20-1

    Combination 4: 7-50-50-11

    Combination 5: 7-20-20-11
    
Here we will use the absolute value of (Sentiment - Mean of Sentiments)

In [22]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense

In [23]:
dataset = pd.read_csv(r"/Users/t_velpac/mission/WorkingCopy/final_csv_files/Abs_Sentiment-Mean.csv")
dataset = dataset.drop(columns=['Date', 'Z_Score_HelpfulVotes', 'Z_Score_Words', 'Z_Score_Paragraphs', 'Sentiment_Polarity'])
cols = (list(dataset.columns))
cols.remove('Helpful_Votes')
cols.append('Helpful_Votes')
dataset = dataset[cols]
dataset.head()

,Stars,Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Absolute(Sentiment-Mean),Helpful_Votes
0,4,268,8,14,3,90,166.125000,0.118601,146
1,1,130,2,2,2,94,352.500000,0.140716,80
2,1,110,1,0,3,90,556.000000,0.173722,80
3,5,110,3,4,3,92,188.666667,0.125016,1563
4,1,39,1,0,3,96,216.000000,0.431510,53


In [3]:
""" Splitting the independent variables from the dependent variable """
X = dataset.iloc[:,0:-1].values
y = dataset.iloc[:,-1].values

In [4]:
"""Splitting the data into training data and testing data"""
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

"""Scaling the data using StandardScaler from sklearn package"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [5]:
""" Building a 3 layer ANN with high number of nodes """

regressor = Sequential()
regressor.add(Dense(100, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor.add(Dense(100, kernel_initializer = 'normal', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
156392/156392 [==============================] - 27s 173us/step - loss: 15569.1812
Epoch 2/100
156392/156392 [==============================] - 29s 182us/step - loss: 15559.1189
Epoch 3/100
156392/156392 [==============================] - 29s 185us/step - loss: 15556.8517
Epoch 4/100
156392/156392 [==============================] - 28s 177us/step - loss: 15561.5714
Epoch 5/100
156392/156392 [==============================] - 27s 171us/step - loss: 15550.6663
Epoch 6/100
156392/156392 [==============================] - 26s 167us/step - loss: 15543.9896
Epoch 7/100
156392/156392 [==============================] - 29s 182us/step - loss: 15549.7957
Epoch 8/100
156392/156392 [==============================] - 27s 172us/step - loss: 15554.0958
Epoch 9/100
156392/156392 [==============================] - 27s 170us/step - loss: 15545.8330
Epoch 10/100
156392/15639

156392/156392 [==============================] - 35s 225us/step - loss: 15540.2752
Epoch 74/100
156392/156392 [==============================] - 39s 251us/step - loss: 15541.3206- ETA: 0s - loss: 15637
Epoch 75/100
156392/156392 [==============================] - 36s 233us/step - loss: 15547.94253s - loss: 1 
Epoch 76/100
156392/156392 [==============================] - 35s 227us/step - loss: 15544.7157
Epoch 77/100
156392/156392 [==============================] - 36s 228us/step - loss: 15539.3504
Epoch 78/100
156392/156392 [==============================] - 36s 227us/step - loss: 15539.9011
Epoch 79/100
156392/156392 [==============================] - 37s 235us/step - loss: 15546.5212
Epoch 80/100
156392/156392 [==============================] - 42s 268us/step - loss: 15538.5257
Epoch 81/100
156392/156392 [==============================] - 42s 266us/step - loss: 15538.25861s - loss: 15746.553 - 
Epoch 82/100
156392/156392 [==============================] - 42s 267us/step - loss: 15556

In [6]:
from sklearn.metrics import r2_score

In [7]:
""" Predicting the values for Helpful Votes on the test data """
y_pred = regressor.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred = y_pred.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred}
y_compare = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare['Mean Squared Error'] = (np.diff(y_compare.values) ** 2)
y_compare['Mean Squared Error'] = np.mean(y_compare['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare['Root Mean Squared Error'] = y_compare['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare['Mean Abs. Error'] = np.mean(abs(y_compare['Actual Values'] - y_compare['Predicted Values']))

# Calculating R-Squared value
y_compare['R_Squared'] = r2_score(y_test, y_pred)

y_compare

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,30.387802,19624.158265,140.086253,9.010501,0.000766
1,0,2.406342,19624.158265,140.086253,9.010501,0.000766
2,0,3.588143,19624.158265,140.086253,9.010501,0.000766
3,0,7.181384,19624.158265,140.086253,9.010501,0.000766
4,0,5.383566,19624.158265,140.086253,9.010501,0.000766
5,0,4.562929,19624.158265,140.086253,9.010501,0.000766
6,3,6.486474,19624.158265,140.086253,9.010501,0.000766
7,10,3.676600,19624.158265,140.086253,9.010501,0.000766
8,0,3.702975,19624.158265,140.086253,9.010501,0.000766
9,0,4.057733,19624.158265,140.086253,9.010501,0.000766


In [8]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train = regressor.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train = y_pred_train.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train }
y_compare_train = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train['Mean Squared Error'] = (np.diff(y_compare_train.values) ** 2)
y_compare_train['Mean Squared Error'] = np.mean(y_compare_train['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train['Root Mean Squared Error'] = y_compare_train['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train['Mean Abs. Error'] = np.mean(abs(y_compare_train['Actual Values(Training)'] - y_compare_train['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train['R_Squared'] = r2_score(y_train, y_pred_train)

y_compare_train

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,5.099553,15559.834997,124.739068,8.984964,0.002636
1,0,22.811249,15559.834997,124.739068,8.984964,0.002636
2,0,8.460062,15559.834997,124.739068,8.984964,0.002636
3,0,3.472726,15559.834997,124.739068,8.984964,0.002636
4,0,4.998594,15559.834997,124.739068,8.984964,0.002636
5,22,3.821988,15559.834997,124.739068,8.984964,0.002636
6,1,5.601140,15559.834997,124.739068,8.984964,0.002636
7,0,6.286368,15559.834997,124.739068,8.984964,0.002636
8,0,3.818510,15559.834997,124.739068,8.984964,0.002636
9,0,5.301939,15559.834997,124.739068,8.984964,0.002636


In [9]:
""" Building a 3 layer ANN with 2 layers of 4 nodes """

regressor_1 = Sequential()
regressor_1.add(Dense(4, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_1.add(Dense(4, kernel_initializer = 'normal', activation = 'relu'))
regressor_1.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_1.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 42s 267us/step - loss: 15551.6423
Epoch 2/100
156392/156392 [==============================] - 40s 256us/step - loss: 15543.92601s - loss: 15493. - ETA: 0s 
Epoch 3/100
156392/156392 [==============================] - 41s 260us/step - loss: 15544.3714
Epoch 4/100
156392/156392 [==============================] - 43s 275us/step - loss: 15541.6088
Epoch 5/100
156392/156392 [==============================] - 38s 246us/step - loss: 15537.21310s - loss: 15
Epoch 6/100
156392/156392 [==============================] - 42s 268us/step - loss: 15544.9526TA:
Epoch 7/100
156392/156392 [==============================] - 42s 266us/step - loss: 15541.527212s - loss: 13330. - ETA: 12s - loss: 1328 - ETA: 11s - loss: 1
Epoch 8/100
156392/156392 [==============================] - 36s 230us/step - loss: 15542.90723s - loss: - E - ETA: 2s - lo - ETA: 0s - loss: 15548.068
Epoch 9/100
156392/156392 [==============================] - 36s 228us/step 

156392/156392 [==============================] - 35s 221us/step - loss: 15548.2542
Epoch 78/100
156392/156392 [==============================] - 34s 218us/step - loss: 15545.1190
Epoch 79/100
156392/156392 [==============================] - 35s 222us/step - loss: 15537.2162
Epoch 80/100
156392/156392 [==============================] - 34s 219us/step - loss: 15551.24240s - loss: 15719. - ETA: 0s - loss: 1561
Epoch 81/100
156392/156392 [==============================] - 34s 220us/step - loss: 15537.3420
Epoch 82/100
156392/156392 [==============================] - 35s 223us/step - loss: 15538.5395
Epoch 83/100
156392/156392 [==============================] - 35s 221us/step - loss: 15538.9407 - ETA: - ETA: 0s -
Epoch 84/100
156392/156392 [==============================] - 34s 217us/step - loss: 15535.3465
Epoch 85/100
156392/156392 [==============================] - 35s 221us/step - loss: 15539.9696
Epoch 86/100
156392/156392 [==============================] - 34s 218us/step - loss: 15543

In [10]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_1 = regressor_1.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_1 = y_pred_1.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_1}
y_compare_1 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_1['Mean Squared Error'] = (np.diff(y_compare_1.values) ** 2)
y_compare_1['Mean Squared Error'] = np.mean(y_compare_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_1['Root Mean Squared Error'] = y_compare_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_1['Mean Abs. Error'] = np.mean(abs(y_compare_1['Actual Values'] - y_compare_1['Predicted Values']))

# Calculating R-Squared value
y_compare_1['R_Squared'] = r2_score(y_test, y_pred_1)

y_compare_1

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,13.969353,19590.565684,139.966302,6.799869,0.002476
1,0,1.489848,19590.565684,139.966302,6.799869,0.002476
2,0,1.489848,19590.565684,139.966302,6.799869,0.002476
3,0,1.489848,19590.565684,139.966302,6.799869,0.002476
4,0,3.200263,19590.565684,139.966302,6.799869,0.002476
5,0,1.489848,19590.565684,139.966302,6.799869,0.002476
6,3,11.440790,19590.565684,139.966302,6.799869,0.002476
7,10,4.496403,19590.565684,139.966302,6.799869,0.002476
8,0,1.489848,19590.565684,139.966302,6.799869,0.002476
9,0,3.523758,19590.565684,139.966302,6.799869,0.002476


In [11]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_1 = regressor_1.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_1 = y_pred_train_1.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_1 }
y_compare_train_1 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_1['Mean Squared Error'] = (np.diff(y_compare_train_1.values) ** 2)
y_compare_train_1['Mean Squared Error'] = np.mean(y_compare_train_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_1['Root Mean Squared Error'] = y_compare_train_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_1['Mean Abs. Error'] = np.mean(abs(y_compare_train_1['Actual Values(Training)'] - y_compare_train_1['Predicted Values(Training)']))


# Calculating R-Squared value
y_compare_train_1['R_Squared'] = r2_score(y_train, y_pred_train_1)

y_compare_train_1

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,1.767547,15527.20581,124.608209,6.817839,0.004728
1,0,45.563862,15527.20581,124.608209,6.817839,0.004728
2,0,1.489848,15527.20581,124.608209,6.817839,0.004728
3,0,1.975131,15527.20581,124.608209,6.817839,0.004728
4,0,1.583217,15527.20581,124.608209,6.817839,0.004728
5,22,2.678887,15527.20581,124.608209,6.817839,0.004728
6,1,1.489848,15527.20581,124.608209,6.817839,0.004728
7,0,1.489848,15527.20581,124.608209,6.817839,0.004728
8,0,1.489848,15527.20581,124.608209,6.817839,0.004728
9,0,2.908286,15527.20581,124.608209,6.817839,0.004728


In [12]:
"""


Now, we will train an ANN with a very high number of nodes. And comparitively more layers.

8-50-100-200-100-50-20-1
"""

'\n\n\nNow, we will train an ANN with a very high number of nodes. And comparitively more layers.\n\n8-50-100-200-100-50-20-1\n'

In [13]:
""" Building a deep and wide ANN with high number of nodes """

from keras.layers.advanced_activations import LeakyReLU
regressor_2 = Sequential()
regressor_2.add(Dense(50, kernel_initializer = 'normal',input_dim = 8))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(200, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(50, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(20, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_2.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 56s 361us/step - loss: 15582.82703s - loss: 16317.6 - 
Epoch 2/100
156392/156392 [==============================] - 56s 359us/step - loss: 15564.95220s - lo
Epoch 3/100
156392/156392 [==============================] - 55s 354us/step - loss: 15566.28143 - ETA:  - ETA: 1s - l - ETA: 0s -
Epoch 4/100
156392/156392 [==============================] - 55s 354us/step - loss: 15583.38280s - loss: 15599.97
Epoch 5/100
156392/156392 [==============================] - 56s 356us/step - loss: 53171.9485
Epoch 6/100
156392/156392 [==============================] - 56s 356us/step - loss: 16286.7922
Epoch 7/100
156392/156392 [==============================] - 56s 356us/step - loss: 3600900.3396
Epoch 8/100
156392/156392 [==============================] - 56s 356us/step - loss: 15547.3839
Epoch 9/100
156392/156392 [==============================] - 56s 356us/step - loss: 15644.1800 - E - ETA: 0s 
Epoch 10/100
156392/156392 [==================

156392/156392 [==============================] - 61s 393us/step - loss: 7864525.5963
Epoch 79/100
156392/156392 [==============================] - 61s 391us/step - loss: 21084.3842
Epoch 80/100
156392/156392 [==============================] - 61s 392us/step - loss: 89152.0816
Epoch 81/100
156392/156392 [==============================] - 61s 388us/step - loss: 420429213.8734
Epoch 82/100
156392/156392 [==============================] - 61s 389us/step - loss: 15812.8145
Epoch 83/100
156392/156392 [==============================] - 61s 388us/step - loss: 15674.10761s - loss: 15 - ETA: 0s - lo
Epoch 84/100
156392/156392 [==============================] - 61s 390us/step - loss: 4867900.2969
Epoch 85/100
156392/156392 [==============================] - 61s 388us/step - loss: 16113.3393 ETA: 12s - loss: 15  - ETA: 9s - ETA: 4s - loss: 16700.08 - ETA:  -  - ETA: 0s - loss: 16167
Epoch 86/100
156392/156392 [==============================] - 61s 389us/step - loss: 15375063004.43435s - loss: 1675

In [14]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_2 = regressor_2.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_2 = y_pred_2.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_2}
y_compare_2 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_2['Mean Squared Error'] = (np.diff(y_compare_2.values) ** 2)
y_compare_2['Mean Squared Error'] = np.mean(y_compare_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_2['Root Mean Squared Error'] = y_compare_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_2['Mean Abs. Error'] = np.mean(abs(y_compare_2['Actual Values'] - y_compare_2['Predicted Values']))

# Calculating R-Squared value
y_compare_2['R_Squared'] = r2_score(y_test, y_pred_2)

y_compare_2

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,12.235488,19622.788216,140.081363,6.592961,0.000835
1,0,2.074115,19622.788216,140.081363,6.592961,0.000835
2,0,2.720796,19622.788216,140.081363,6.592961,0.000835
3,0,3.722950,19622.788216,140.081363,6.592961,0.000835
4,0,2.330537,19622.788216,140.081363,6.592961,0.000835
5,0,2.315578,19622.788216,140.081363,6.592961,0.000835
6,3,6.691790,19622.788216,140.081363,6.592961,0.000835
7,10,2.471771,19622.788216,140.081363,6.592961,0.000835
8,0,2.348719,19622.788216,140.081363,6.592961,0.000835
9,0,2.500778,19622.788216,140.081363,6.592961,0.000835


In [15]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_2 = regressor_2.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_2 = y_pred_train_2.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_2 }
y_compare_train_2 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_2['Mean Squared Error'] = (np.diff(y_compare_train_2.values) ** 2)
y_compare_train_2['Mean Squared Error'] = np.mean(y_compare_train_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_2['Root Mean Squared Error'] = y_compare_train_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_2['Mean Abs. Error'] = np.mean(abs(y_compare_train_2['Actual Values(Training)'] - y_compare_train_2['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train_2['R_Squared'] = r2_score(y_train, y_pred_train_2)

y_compare_train_2

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,2.237700,15563.932894,124.755492,6.600316,0.002374
1,0,21.219759,15563.932894,124.755492,6.600316,0.002374
2,0,3.747445,15563.932894,124.755492,6.600316,0.002374
3,0,2.219976,15563.932894,124.755492,6.600316,0.002374
4,0,2.231816,15563.932894,124.755492,6.600316,0.002374
5,22,2.352762,15563.932894,124.755492,6.600316,0.002374
6,1,3.010464,15563.932894,124.755492,6.600316,0.002374
7,0,2.774359,15563.932894,124.755492,6.600316,0.002374
8,0,2.625829,15563.932894,124.755492,6.600316,0.002374
9,0,2.243192,15563.932894,124.755492,6.600316,0.002374


In [16]:
""" Building a 3 layer ANN with 2 layers of 50 nodes """

regressor_3 = Sequential()
regressor_3.add(Dense(50, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_3.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
regressor_3.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_3.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 39s 247us/step - loss: 15559.1256
Epoch 2/100
156392/156392 [==============================] - 39s 247us/step - loss: 15555.3248
Epoch 3/100
156392/156392 [==============================] - 39s 247us/step - loss: 15553.1652
Epoch 4/100
156392/156392 [==============================] - 39s 247us/step - loss: 15545.6388
Epoch 5/100
156392/156392 [==============================] - 39s 249us/step - loss: 15550.1138
Epoch 6/100
156392/156392 [==============================] - 39s 249us/step - loss: 15551.92551s -  - ETA: 0s - los
Epoch 7/100
156392/156392 [==============================] - 39s 251us/step - loss: 15545.1406
Epoch 8/100
156392/156392 [==============================] - 40s 253us/step - loss: 15539.7963A: 3s 
Epoch 9/100
156392/156392 [==============================] - 39s 251us/step - loss: 15541.24661s - loss: 15713.62 - ETA: 1s - loss: 15673. - ETA: 0s - loss: 15561.86
Epoch 10/100
156392/156392 [===================

156392/156392 [==============================] - 34s 219us/step - loss: 15550.79595s -  - E
Epoch 76/100
156392/156392 [==============================] - 34s 219us/step - loss: 15540.0237
Epoch 77/100
156392/156392 [==============================] - 35s 221us/step - loss: 15534.8268
Epoch 78/100
156392/156392 [==============================] - 34s 220us/step - loss: 15544.54829s - loss: 16758. - ETA: - ETA
Epoch 79/100
156392/156392 [==============================] - 35s 221us/step - loss: 15534.58081s - lo - ETA: 0s - loss
Epoch 80/100
156392/156392 [==============================] - 34s 219us/step - loss: 15537.2440
Epoch 81/100
156392/156392 [==============================] - 35s 226us/step - loss: 15548.0080
Epoch 82/100
156392/156392 [==============================] - 34s 220us/step - loss: 15538.5452
Epoch 83/100
156392/156392 [==============================] - 35s 221us/step - loss: 15545.9172
Epoch 84/100
156392/156392 [==============================] - 34s 218us/step - loss: 1

In [17]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_3 = regressor_3.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_3 = y_pred_3.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_3}
y_compare_3 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_3['Mean Squared Error'] = (np.diff(y_compare_3.values) ** 2)
y_compare_3['Mean Squared Error'] = np.mean(y_compare_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_3['Root Mean Squared Error'] = y_compare_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_3['Mean Abs. Error'] = np.mean(abs(y_compare_3['Actual Values'] - y_compare_3['Predicted Values']))

# Calculating R-Squared value
y_compare_3['R_Squared'] = r2_score(y_test, y_pred_3)

y_compare_3

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,21.042082,19619.657249,140.070187,8.608414,0.000995
1,0,1.360035,19619.657249,140.070187,8.608414,0.000995
2,0,2.912747,19619.657249,140.070187,8.608414,0.000995
3,0,7.655807,19619.657249,140.070187,8.608414,0.000995
4,0,2.341278,19619.657249,140.070187,8.608414,0.000995
5,0,2.203229,19619.657249,140.070187,8.608414,0.000995
6,3,7.529213,19619.657249,140.070187,8.608414,0.000995
7,10,8.185163,19619.657249,140.070187,8.608414,0.000995
8,0,0.988979,19619.657249,140.070187,8.608414,0.000995
9,0,3.823816,19619.657249,140.070187,8.608414,0.000995


In [18]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_3 = regressor_3.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_3 = y_pred_train_3.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_3 }
y_compare_train_3 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_3['Mean Squared Error'] = (np.diff(y_compare_train_3.values) ** 2)
y_compare_train_3['Mean Squared Error'] = np.mean(y_compare_train_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_3['Root Mean Squared Error'] = y_compare_train_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_3['Mean Abs. Error'] = np.mean(abs(y_compare_train_3['Actual Values(Training)'] - y_compare_train_3['Predicted Values(Training)']))


# Calculating R-Squared value
y_compare_train_3['R_Squared'] = r2_score(y_train, y_pred_train_3)

y_compare_train_3

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,2.546804,15555.05151,124.719892,8.601659,0.002943
1,0,19.968065,15555.05151,124.719892,8.601659,0.002943
2,0,5.162373,15555.05151,124.719892,8.601659,0.002943
3,0,5.815307,15555.05151,124.719892,8.601659,0.002943
4,0,3.863237,15555.05151,124.719892,8.601659,0.002943
5,22,9.324267,15555.05151,124.719892,8.601659,0.002943
6,1,9.715837,15555.05151,124.719892,8.601659,0.002943
7,0,6.347829,15555.05151,124.719892,8.601659,0.002943
8,0,0.933609,15555.05151,124.719892,8.601659,0.002943
9,0,2.726391,15555.05151,124.719892,8.601659,0.002943


In [19]:
""" Building a 3 layer ANN with 2 layers of 20 nodes """

regressor_4 = Sequential()
regressor_4.add(Dense(20, kernel_initializer = 'normal',activation = 'relu',input_dim = 8))
regressor_4.add(Dense(20, kernel_initializer = 'normal', activation = 'relu'))
regressor_4.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_4.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_4.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
156392/156392 [==============================] - 36s 228us/step - loss: 15548.6565
Epoch 2/100
156392/156392 [==============================] - 35s 224us/step - loss: 15546.5996
Epoch 3/100
156392/156392 [==============================] - 35s 226us/step - loss: 15546.1958
Epoch 4/100
156392/156392 [==============================] - 35s 222us/step - loss: 15546.25752s - los - ETA: 1s - loss: 15892.89 
Epoch 5/100
156392/156392 [==============================] - 35s 224us/step - loss: 15539.3635
Epoch 6/100
156392/156392 [==============================] - 35s 223us/step - loss: 15540.1737 - ETA: 0s - loss: 1
Epoch 7/100
156392/156392 [==============================] - 35s 222us/step - loss: 15537.3990
Epoch 8/100
156392/156392 [==============================] - 35s 226us/step - loss: 15546.7362
Epoch 9/100
156392/156392 [==============================] - 34s 220us/step - loss: 15541.80080s - 
Epoch 10/100
156392/156392 [==============================] - 35s 224us/step - loss:

156392/156392 [==============================] - 35s 225us/step - loss: 15542.6600
Epoch 79/100
156392/156392 [==============================] - 35s 222us/step - loss: 15542.39741s - lo - ETA: 0s - loss: 157 - ETA: 0s - loss: 15618.06 - ETA: 0s - loss: 1
Epoch 80/100
156392/156392 [==============================] - 35s 222us/step - loss: 15540.9377
Epoch 81/100
156392/156392 [==============================] - 34s 219us/step - loss: 15550.5378
Epoch 82/100
156392/156392 [==============================] - 35s 223us/step - loss: 15547.0348
Epoch 83/100
156392/156392 [==============================] - 35s 221us/step - loss: 15541.29423s  - ETA: 0s - los
Epoch 84/100
156392/156392 [==============================] - 34s 220us/step - loss: 15548.99710s - l
Epoch 85/100
156392/156392 [==============================] - 35s 222us/step - loss: 15537.0221
Epoch 86/100
156392/156392 [==============================] - 35s 222us/step - loss: 15546.54365s - loss: 12 - E - ETA: 0s - los
Epoch 87/100
15

In [20]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_4 = regressor_4.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_4 = y_pred_4.reshape(39099,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_4}
y_compare_4 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_4['Mean Squared Error'] = (np.diff(y_compare_4.values) ** 2)
y_compare_4['Mean Squared Error'] = np.mean(y_compare_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_4['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_4['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

# Calculating R-Squared value
y_compare_4['R_Squared'] = r2_score(y_test, y_pred_4)

y_compare_4

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,14.366382,19585.182103,139.947069,6.67853,0.00275
1,0,0.277088,19585.182103,139.947069,6.67853,0.00275
2,0,0.648492,19585.182103,139.947069,6.67853,0.00275
3,0,2.535481,19585.182103,139.947069,6.67853,0.00275
4,0,1.655498,19585.182103,139.947069,6.67853,0.00275
5,0,1.079884,19585.182103,139.947069,6.67853,0.00275
6,3,12.766371,19585.182103,139.947069,6.67853,0.00275
7,10,2.356017,19585.182103,139.947069,6.67853,0.00275
8,0,-0.066197,19585.182103,139.947069,6.67853,0.00275
9,0,1.651430,19585.182103,139.947069,6.67853,0.00275


In [21]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_4 = regressor_4.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_4 = y_pred_train_4.reshape(156392,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_4 }
y_compare_train_4 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_4['Mean Squared Error'] = (np.diff(y_compare_train_4.values) ** 2)
y_compare_train_4['Mean Squared Error'] = np.mean(y_compare_train_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_4['Root Mean Squared Error'] = y_compare_train_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_4['Mean Abs. Error'] = np.mean(abs(y_compare_train_4['Actual Values(Training)'] - y_compare_train_4['Predicted Values(Training)']))

# Calculating R-Squared value
y_compare_train_4['R_Squared'] = r2_score(y_train, y_pred_train_4)

y_compare_train_4

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error,R_Squared
0,0,1.368471,15534.336411,124.636818,6.690266,0.004271
1,0,29.252314,15534.336411,124.636818,6.690266,0.004271
2,0,1.954439,15534.336411,124.636818,6.690266,0.004271
3,0,2.231013,15534.336411,124.636818,6.690266,0.004271
4,0,1.340164,15534.336411,124.636818,6.690266,0.004271
5,22,2.346267,15534.336411,124.636818,6.690266,0.004271
6,1,2.385730,15534.336411,124.636818,6.690266,0.004271
7,0,2.215977,15534.336411,124.636818,6.690266,0.004271
8,0,-0.048419,15534.336411,124.636818,6.690266,0.004271
9,0,2.298059,15534.336411,124.636818,6.690266,0.004271
